# Dataset

Let's try to generate our own dataset in a form of knowledge base.

1. Retrieve a bunch of texts
2. Get tickers
3. Lookup tickers in Wikidata
4. Map 'em with NERS

In [2]:
import sys
type(sys.path)

list

In [3]:
sys.path.append("/mnt/codeholder/code/python-playground/app_noisemon/noisemon")

## 1. Data import

In [4]:
import json
from pathlib import Path

In [12]:
input_path = Path("../data/01-filtered/market_twits_with_tickers.json")
data = json.loads(input_path.read_text())

In [4]:
len(data)

NameError: name 'data' is not defined

In [14]:
data[38]

'AFLT\nРостех» планирует продать свою долю в «Аэрофлоте» (3,5%) целиком, а не частями.\nПакет будет выставлен на продажу, когда цена акций вырастет хотя бы до 182 руб. за бумагу, передает ТАСС.'

## 2. Extract tickers

In [16]:
import reticker
extractor = reticker.TickerExtractor()

In [17]:
dataset = []
for text in data:
    dataset.append({
        "text": text, 
        "tickers": extractor.extract(text)
    })

## 3. Lookup tickers in wikidata

In [18]:
dataset[56]

{'text': 'ETH\nМартин Свенде, разработчик службы безопасности Эфириума - "Чтобы разморозить счета Parity, понадобится новый хардфорк Эфириума"',
 'tickers': ['ETH']}

In [184]:
from importlib import reload

In [185]:
import data_processing.wikidata as ddd
reload(ddd)

<module 'data_processing.wikidata' from '/mnt/codeholder/code/python-playground/app_noisemon/noisemon/data_processing/wikidata.py'>

In [186]:
from data_processing.wikidata import Wikidata
from functools import lru_cache
from tqdm import tqdm
import time

In [187]:
wd = Wikidata()

In [188]:
def throttle(func):
    __last_call_start = time.time()
    __timeout = 2
    def inner(*args):
        # record time since last launch
        nonlocal __last_call_start
        timeout = __last_call_start + __timeout - time.time()
        # update last launch
        
        __last_call_start = time.time()
#         print("Timeout: ", timeout)
        if timeout > 0:
            time.sleep(timeout)
            
        return func(*args)
    
    return inner 

In [189]:
@throttle
def r():
    return "ddd"

In [190]:
r()

'ddd'

In [191]:
@lru_cache(maxsize=None)
@throttle
def get_company(ticker: str):
    return wd.lookup_companies_by_ticker(ticker)

In [199]:
ticker_not_found = []
dataset_with_companies = []
for chunk in tqdm(dataset):
    companies = []
    for ticker in set(chunk["tickers"]):
        company_candidates = get_company(ticker)
        if company_candidates:
            companies += company_candidates
    result = {**chunk, "companies" : companies}
    if companies:
        dataset_with_companies.append(result)
    else:
        ticker_not_found.append(result)
        
print(f"{len(dataset_with_companies)} texts with companies, {len(ticker_not_found)} without")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88357/88357 [00:00<00:00, 302476.33it/s]

55758 texts with companies, 32599 without


In [203]:
ticker_not_found[900]

{'text': 'НЕФТЬ - DUMB MONEY - лонги в фонде USO - 94% всех фондов торгующими USO в лонгах!!! Рекорд с 2007. Критическая зона. Статистика против лонгов.',
 'tickers': ['DUMB', 'USO'],
 'companies': []}

In [201]:
dataset_with_companies[1005]

{'text': 'BTC vs GOLD',
 'tickers': ['BTC', 'GOLD'],
 'companies': [{'id': {'type': 'uri',
    'value': 'http://www.wikidata.org/entity/Q131723'},
   'idLabel': {'xml:lang': 'ru', 'type': 'literal', 'value': 'биткойн'}},
  {'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1961738'},
   'idLabel': {'xml:lang': 'ru',
    'type': 'literal',
    'value': 'Amex Gold BUGS Index'}},
  {'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1145004'},
   'idLabel': {'xml:lang': 'en',
    'type': 'literal',
    'value': 'Randgold Resources'}},
  {'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q96100267'},
   'idLabel': {'xml:lang': 'en',
    'type': 'literal',
    'value': 'Visi Telekomunikasi Infrastruktur'}}]}

In [5]:
output_path = Path("../data/03-populated")

In [210]:
with open(output_path / "market_twits_with_companies_by_ticker.json", "w") as fout:
    json.dump(dataset_with_companies, fout, ensure_ascii=False)

In [211]:
with open(output_path / "market_twits_with_unknown_tickers.json", "w") as fout:
    json.dump(ticker_not_found, fout, ensure_ascii=False)

In [209]:
# As long as the function was cached, no extra requests needed
ticker_map = {}
for chunk in tqdm(dataset):
    for ticker in set(chunk["tickers"]):
        ticker_map[ticker] = get_company(ticker)
        
print(f"{len(ticker_map)} tickers in total")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88357/88357 [00:00<00:00, 1193135.11it/s]

11347 tickers in total


In [212]:
with open(output_path / "ticker_to_response_map.json", "w") as fout:
    json.dump(ticker_map, fout, ensure_ascii=False)

## 4. Extract NERS

1. Load populated data
2. Extract ORGanization for each text
3. Match 'em w/found companys (field :value)

In [13]:
from tqdm import tqdm
import spacy
import regex
nlp_ru = spacy.load("ru_core_news_lg")
nlp_en = spacy.load("en_core_web_lg")

In [25]:
regex.search("[А-яЁё]","ffff")

In [79]:
with open(output_path / "market_twits_with_companies_by_ticker.json", "r") as fin:
    dataset_with_companies = json.load(fin)

In [80]:
dataset_with_companies[34982]

{'text': '💥🇺🇸 #INFY  Infosys price target  raised  to $18 from $13 at BMO Capital',
 'tickers': ['INFY', 'BMO'],
 'companies': [{'id': {'type': 'uri',
    'value': 'http://www.wikidata.org/entity/Q806693'},
   'idLabel': {'xml:lang': 'ru',
    'type': 'literal',
    'value': 'Bank of Montreal'}},
  {'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1662481'},
   'idLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'IHS Markit'}},
  {'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q26989'},
   'idLabel': {'xml:lang': 'ru', 'type': 'literal', 'value': 'Infosys'}}]}

In [81]:
for chunk in tqdm(dataset_with_companies):
    text = chunk["text"]
    if regex.search("[А-яЁё]", text):
        doc = nlp_ru(text)
    else:
        doc = nlp_en(text)
    entities = []
    for entity in doc.ents:
        if entity.label_ == "ORG":
            entities.append(entity)
    chunk["entities"] = entities

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 55758/55758 [11:36<00:00, 80.11it/s]


## 5. Map NERS and Organizations

In [82]:
from difflib import SequenceMatcher
def get_distance(A, B) -> float:
    return SequenceMatcher(None, A, B).ratio()

In [83]:
def match(A, B) -> bool:
    A, B = A.lower(), B.lower()
    if (A in B) or (B in A):
        return True
    elif get_distance(A, B) > 0.83:
        return True
    
    return False

In [88]:
for chunk in tqdm(dataset_with_companies):
    target = []
    for entity in chunk["entities"]:
        entity_text = entity.lemma_
        matched_company = None
        for company in chunk["companies"]:
            company_name = company["idLabel"]["value"]
            if match(entity_text, company_name):
                matched_company = company
        target.append((entity.start_char, entity.end_char, matched_company))
    chunk["target"] = target

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 55758/55758 [00:06<00:00, 8982.18it/s]


In [94]:
dataset_with_companies[955]

{'text': 'TRCN\nГруппа UCL Владимира Лисина просит министерство экономического развития РФ "жестко ограничить инвестиционную активность" ПАО "Трансконтейнер" на период продажи его контрольного пакета акций, принадлежащего ОАО "Российские железные дороги".',
 'tickers': ['TRCN', 'UCL'],
 'companies': [{'id': {'type': 'uri',
    'value': 'http://www.wikidata.org/entity/Q157062'},
   'idLabel': {'xml:lang': 'ru', 'type': 'literal', 'value': 'Unilever'}}],
 'entities': [TRCN,
  UCL,
  ПАО "Трансконтейнер",
  ОАО "Российские железные дороги"],
 'target': [(0, 5, None), (12, 15, None), (126, 146, None), (211, 243, None)]}

In [76]:
dataset_with_companies[56]["entities"][0].lemma_

'mgnt'

In [47]:
entity.text

'Nvidia'

In [46]:
entity.end_char

469

In [53]:
company

{'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1023876'},
 'idLabel': {'xml:lang': 'ru', 'type': 'literal', 'value': 'CME Group'}}

In [62]:
dataset_with_companies[36]

{'text': 'HYDR\nКомитет Госдумы по бюджету на заседании в понедельник поддержал правительственные поправки в проект федерального бюджета на 2018-2020 годы ко второму чтению об увеличении уставного капитала "РусГидро" (MOEX: HYDR) в 2018 году на 1 млрд рублей, в 2019 году - на 3 млрд рублей, в 2020 году - на 6 млрд рублей. \nКак говорится в поправках, средства будут направлены на модернизацию и новое строительство электросетевых объектов.',
 'tickers': ['HYDR', 'MOEX'],
 'companies': [{'id': {'type': 'uri',
    'value': 'http://www.wikidata.org/entity/Q2035424'},
   'idLabel': {'xml:lang': 'ru', 'type': 'literal', 'value': 'РусГидро'}},
  {'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2632892'},
   'idLabel': {'xml:lang': 'ru',
    'type': 'literal',
    'value': 'Московская биржа'}}],
 'entities': [заседании в,
  году - на 3,
  году - на,
  на модернизацию,
  новое строительство],
 'target': [(26, 30, None),
  (49, 55, None),
  (96, 102, None),
  (105, 112, None),
  (

In [97]:
with open(output_path / "market_twits_with_companies_and_entities_matched.json", "w") as fout:
    for chunk in dataset_with_companies:
        del chunk["entities"]
    json.dump(dataset_with_companies, fout, ensure_ascii=False)

## 5. Format for LabelStudio

In [11]:
input_path = output_path
output_path = Path("../data/04-vendor_formats")
output_path.exists()

True

In [6]:
with open(input_path / "market_twits_with_companies_and_entities_matched.json", "rb") as fin:
    dataset_with_companies_populated = json.load(fin)

In [7]:
dataset_with_companies_populated[0]

{'text': 'CME Group представила полную спецификацию фьючерсов на биткоин',
 'tickers': ['CME'],
 'companies': [{'id': {'type': 'uri',
    'value': 'http://www.wikidata.org/entity/Q1023876'},
   'idLabel': {'xml:lang': 'ru', 'type': 'literal', 'value': 'CME Group'}}],
 'target': [[0,
   9,
   {'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1023876'},
    'idLabel': {'xml:lang': 'ru', 'type': 'literal', 'value': 'CME Group'}}]]}

In [9]:
for chunk in dataset_with_companies_populated:
    chunk["sidenote"] = "\n".join(list(map(lambda x: f"{x['idLabel']['value']} {x['id']['value']}", chunk["companies"])))

In [10]:
dataset_with_companies_populated[0]

{'text': 'CME Group представила полную спецификацию фьючерсов на биткоин',
 'tickers': ['CME'],
 'companies': [{'id': {'type': 'uri',
    'value': 'http://www.wikidata.org/entity/Q1023876'},
   'idLabel': {'xml:lang': 'ru', 'type': 'literal', 'value': 'CME Group'}}],
 'target': [[0,
   9,
   {'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1023876'},
    'idLabel': {'xml:lang': 'ru', 'type': 'literal', 'value': 'CME Group'}}]],
 'sidenote': 'CME Group http://www.wikidata.org/entity/Q1023876'}

In [12]:
with open(output_path / "data_for_labelstudio.json", "w") as fout:
    json.dump(dataset_with_companies_populated, fout, ensure_ascii=False)

## 5. Form a KnowledgeBase

In [ ]:
from knowledge_base.storage import MyKnowledgeBase
kb = MyKnowledgeBase()